# Retina MNIST notebook

## import libraries

In [26]:
import os
# Set TensorFlow logging to only show errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # This silences INFO and WARNING messages
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Add, Dense, Dropout, Embedding, GlobalAveragePooling1D, Input, Layer, LayerNormalization, MultiHeadAttention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
tf.get_logger().setLevel('ERROR')

try:
  import pennylane as qml
except:
  !pip install pennylane
  import pennylane as qml
from pennylane.operation import Operation

## Import the dataset

In [27]:
# Set the random seed
random = 10 

In [28]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator
import random as rd


In [29]:
def download_and_prepare_dataset(data_info: dict):
    """Utility function to download the dataset.

    Arguments:
        data_info (dict): Dataset metadata.
    """
    data_path = tf.keras.utils.get_file(origin=data_info["url"], md5_hash=data_info["MD5"])

    with np.load(data_path) as data:
        # Get videos
        train_videos = data["train_images"]
        valid_videos = data["val_images"]
        test_videos = data["test_images"]

        # Get labels
        train_labels = data["train_labels"].flatten()
        valid_labels = data["val_labels"].flatten()
        test_labels = data["test_labels"].flatten()

    return (
        (train_videos, train_labels),
        (valid_videos, valid_labels),
        (test_videos, test_labels),
    )


# Get the metadata of the dataset
info = medmnist.INFO["retinamnist"]
# info = medmnist.INFO["retinamnist"]


# Get the dataset
prepared_dataset = download_and_prepare_dataset(info)
(x_train, y_train) = prepared_dataset[0]
(x_val, y_val) = prepared_dataset[1]
(x_test, y_test) = prepared_dataset[2]

In [30]:
import numpy as np
from sklearn.decomposition import PCA
from tensorflow.keras.utils import to_categorical

x_train = x_train/255
x_test = x_test/255

# Expand the dimensions of the images to (28, 28, 1) to represent the grayscale channel explicitly
train_images = np.expand_dims(x_train, -1)
test_images = np.expand_dims(x_test, -1)

# Flatten the images to 2D arrays for PCA
train_images_flat = train_images.reshape(train_images.shape[0], -1)  # Shape: (num_samples, 28*28)
test_images_flat = test_images.reshape(test_images.shape[0], -1)

# Initialize PCA to keep 8 components
pca = PCA(n_components=6)

# Fit PCA on the training images and transform the datasets
train_images = pca.fit_transform(train_images_flat)
test_images = pca.transform(test_images_flat)

# Map the labels 3 -> 0 and 6 -> 1
y_train = np.where(y_train == 0, 0, 1)
y_test = np.where(y_test == 0, 0, 1)

# One-hot encode the labels
train_labels = to_categorical(y_train, 2)
test_labels = to_categorical(y_test, 2)

# Print the shapes of the processed datasets
print("Shape of train images after PCA:", train_labels.shape)
print("Shape of test images after PCA:", test_labels.shape)
print("Shape of train labels:", train_labels.shape)
print("Shape of test labels:", test_labels.shape)


Shape of train images after PCA: (1080, 2)
Shape of test images after PCA: (400, 2)
Shape of train labels: (1080, 2)
Shape of test labels: (400, 2)


In [31]:
unique_labels = np.unique(y_train)
print(f"Unique labels in the training set: {unique_labels}")

Unique labels in the training set: [0 1]


In [32]:
def plot_images(images, labels, num_images=25, figsize=(10,10)):
    grid_size = 5
    plt.figure(figsize=figsize)

    for i in range(num_images):
        plt.subplot(grid_size, grid_size, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap='gray')
        plt.xlabel(f'Label: {labels[i]}')
    plt.show()

In [33]:
def plot_learning_curve(history):
    # Extracting training and validation accuracy
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    # Extracting training and validation loss
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Plotting the accuracy
    plt.figure(figsize=(12, 5))
    
    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(accuracy, label='Training Accuracy')
    plt.plot(val_accuracy, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    # Show the plots
    plt.tight_layout()
    plt.show()


##  Quantum functions

In [34]:
class RBSGate(Operation):
    num_params = 1
    num_wires = 2
    par_domain = 'R'

    def __init__(self, theta, wires):
        super().__init__(theta, wires=wires)
        self.theta = theta

    @staticmethod
    def compute_matrix(theta):
        cos = tf.cos(theta)
        sin = tf.sin(theta)
        return tf.convert_to_tensor([
            [1, 0, 0, 0],
            [0, cos, sin, 0],
            [0, -sin, cos, 0],
            [0, 0, 0, 1]
        ], dtype=tf.float64)

    def adjoint(self):
        return RBSGate(-self.parameters[0], wires=self.wires)

    def label(self, decimals=None, base_label=None, **kwargs):
        theta = self.parameters[0]
        return f"RBS({theta:.2f})"
def convert_array(X):
    alphas = tf.zeros(X.shape[:-1] + (X.shape[-1]-1,), dtype=X.dtype)
    X_normd = tf.linalg.l2_normalize(X, axis=-1)
    for i in range(X.shape[-1]-1):
        prod_sin_alphas = tf.reduce_prod(tf.sin(alphas[..., :i]), axis=-1)
        updated_value = tf.acos(X_normd[..., i] / prod_sin_alphas)
        indices = tf.constant([[i]])
        updates = tf.reshape(updated_value, [1])
        alphas = tf.tensor_scatter_nd_update(alphas, indices, updates)
    return alphas
def vector_loader(alphas, wires=None, is_x=True, is_conjugate=False):
    if wires is None:
        wires = list(range(len(alphas) + 1))
    if is_x and not is_conjugate:
        qml.PauliX(wires=wires[0])
    if is_conjugate:
        for i in range(len(wires) - 2, -1, -1):
            qml.apply(RBSGate(-alphas[i], wires=[wires[i], wires[i+1]]))
    else:
        for i in range(len(wires) - 1):
            qml.apply(RBSGate(alphas[i], wires=[wires[i], wires[i+1]]))
    if is_x and is_conjugate:
        qml.PauliX(wires=wires[0])
def pyramid_circuit(parameters, wires=None):
    if wires is None:
        length = len(qml.device.wires)
    else:
        length = len(wires)

    k = 0

    for i in range(2 * length - 2):
        j = length - abs(length - 1 - i)

        if i % 2:
            for _ in range(j):
                if _ % 2 == 0 and k < (parameters.shape[0]):
                    qml.apply(RBSGate(parameters[k], wires=([wires[_], wires[_ + 1]])))
                    k += 1
        else:
            for _ in range(j):
                if _ % 2 and k < (parameters.shape[0]):
                    qml.apply(RBSGate(parameters[k], wires=([wires[_], wires[_ + 1]])))
                    k += 1

# qOrthNN + Dynamic Dropout

In [35]:
class HybridModel(tf.keras.Model):
    def __init__(self,apply_quantum_dropout):
        super(HybridModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(6, activation='linear', dtype=tf.float64)
        self.quantum_weights = self.add_weight(
            shape=(15,),
            initializer='zeros',
            trainable=True,
            dtype=tf.float32
        )
        self.theta_locked = self.add_weight(
            shape=(15,),
            initializer='zeros',
            trainable=False,
            dtype=tf.float32
        )
        
        self.dev = qml.device('default.qubit.tf', wires=6)

        @qml.qnode(self.dev, interface='tf', diff_method='backprop')
        def quantum_circuit(inputs, weights):
            inputs = tf.cast(inputs, tf.float32)
            weights = tf.cast(weights, tf.float32)
            vector_loader(convert_array(inputs), wires=range(6))
            pyramid_circuit(weights, wires=range(6))
            return [qml.expval(qml.PauliZ(wire)) for wire in range(6)]

        self.quantum_circuit = quantum_circuit
        self.classical_nn_2 = tf.keras.layers.Dense(2, activation='sigmoid', dtype=tf.float64)
        
        # Droppout mask
        self.theta_wire_0 = tf.constant([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1], dtype=tf.int32)
        self.theta_wire_1 = tf.constant([0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], dtype=tf.int32)
        # Only Drop 1 Qubit or 1 Wire 
        self.n_drop = tf.constant(1, dtype=tf.int32)
            
        self.drop_flag = tf.Variable(apply_quantum_dropout, trainable=False)
        
        
    def call(self, inputs):
        inputs = tf.cast(inputs, tf.float64)
        flattened_inputs = self.flatten(inputs)
        classical_output = self.dense(flattened_inputs)
        quantum_outputs = tf.map_fn(
            lambda x: tf.stack(self.quantum_circuit(x, self.quantum_weights)),
            classical_output,
            fn_output_signature=tf.TensorSpec(shape=(6,), dtype=tf.float64)
        )

        # Apply the condition using tf.cond
        quantum_outputs = tf.cond(
            self.drop_flag,
            lambda: tf.concat([
                tf.zeros((tf.shape(quantum_outputs)[0], 1), dtype=tf.float64),  # Create a tensor of zeros
                quantum_outputs[:, 1:]  # Keep the rest of the elements
            ], axis=1),
            lambda: quantum_outputs  # Keep original quantum_outputs if drop_flag is False
        )
        # Handle NaN values in quantum outputs
        quantum_outputs = tf.where(tf.math.is_nan(quantum_outputs), tf.zeros_like(quantum_outputs), quantum_outputs)

        # Combine and process quantum outputs through additional NN layers
        quantum_outputs = tf.reshape(quantum_outputs, [-1, 6])
        nn_output = self.classical_nn_2(quantum_outputs)

        return nn_output
    def train_step(self, data):
        x, y = data  # Unpack the data

        # Lock the dropped gates
        self.theta_locked = tf.identity(self.quantum_weights)

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        gradients = tape.gradient(loss, self.trainable_variables)

        # Sanitize gradients: replace NaNs with zeros
        sanitized_gradients = []
        
        for grad in gradients:
            if grad is not None:
                # Replace NaNs with zeros
                grad = tf.where(tf.math.is_nan(grad), tf.zeros_like(grad), grad)
            sanitized_gradients.append(grad)
           
        # Apply the dropout mask: set the elements to 0 where mask is 1 
        def one_wire_drop():
            quantum_masked_gradients = tf.where(self.theta_wire_0 == 1, 0.0, gradients[0])
            updated_gradients = sanitized_gradients
            updated_gradients[0] = quantum_masked_gradients
            return updated_gradients
        def two_wire_drop():
            quantum_masked_gradients = tf.where(self.theta_wire_0 == 1, 0.0, gradients[0])
            quantum_masked_gradients = tf.where(self.theta_wire_1 == 1, 0.0, quantum_masked_gradients)
            updated_gradients = sanitized_gradients
            updated_gradients[0] = quantum_masked_gradients
            return updated_gradients
        def no_wire_drop():
            sanitized_gradients1 = []
            for grad in gradients:
                if grad is not None:
                    # Replace NaNs with zeros
                    grad = tf.where(tf.math.is_nan(grad), tf.zeros_like(grad), grad)
                sanitized_gradients1.append(grad)
            return sanitized_gradients1
        # Apply tf.cond based on the value of self.n_drop
        sanitized_gradients = tf.cond(
            tf.logical_and(tf.equal(self.n_drop, 1), tf.equal(self.drop_flag, True)),  # Combine conditions
            one_wire_drop,  # If both conditions are true, execute one_wire_drop
            no_wire_drop    # If either condition is false, execute no_wire_drop
        )
        # Apply the sanitized gradients
        self.optimizer.apply_gradients(zip(sanitized_gradients, self.trainable_variables))
            
        # tf.print("Drop:", self.drop_flag)
        # tf.print(self.theta_locked, summarize=-1)
        # tf.print(self.quantum_weights, summarize=-1)
        #make weights that are nan 0
        for var in self.trainable_variables:
            # Create a mask where NaNs are present
            nan_mask = tf.math.is_nan(var)
            # Replace NaNs with zeros
            sanitized_var = tf.where(nan_mask, tf.zeros_like(var), var)
            # Assign the sanitized variable back to the model
            var.assign(sanitized_var)
        # Update metrics
        self.compiled_metrics.update_state(y, y_pred)
        # tf.print(self.quantum_weights, summarize=-1)

        # Return a dictionary of metric results
        return {m.name: m.result() for m in self.metrics}

In [36]:
# Define a custom callback to log and update the learning rate
class LearningRateLogger(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.new_lr = None

    def on_epoch_end(self, epoch, logs=None):
        # Get the current learning rate
        current_lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Store the learning rate for the next iteration
        self.new_lr = current_lr

# Monte Carlo Inspection

In [24]:
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf

all_train_auc = []  # List to store training AUC for plotting
all_val_auc = []  # List to store validation AUC for plotting
all_losses = []  # For training loss
all_val_losses = []  # For validation loss
all_train_acc = []  # For training accuracy
all_val_acc = []  # For validation accuracy
seeds = []
n_simulations = 10 
# Monte Carlo Resampling
for _ in range(n_simulations):
    if os.path.exists('/home/HardDisk/quang_nguyen/quantum_ml/experiment_dropout/quantum_weights3.weights.h5'):
        os.remove('/home/HardDisk/quang_nguyen/quantum_ml/experiment_dropout/quantum_weights3.weights.h5')
    rd.seed(random)
    np.random.seed(random)
    tf.random.set_seed(random)
    qml.numpy.random.seed(random)
    print('Training with random seed:', random)
    initial_lr = 0.3
    seed_losses = []
    seed_val_losses = []
    seed_train_acc = []
    seed_val_acc = []
    seed_train_auc = []  # List to store training AUC for each epoch
    seed_val_auc = []  # List to store validation AUC for each epoch

    for iteration in range(1, 10):
        print(f"Epochs {iteration}")

        # Switch dropout flag randomly
        if rd.random() <= 0.5 and iteration != 1:
            drop_flag = True
            print("Applying Quantum Dropout")
        else:
            drop_flag = False
            print("Not Applying Quantum Dropout")

        # Create the model
        model = HybridModel(apply_quantum_dropout=drop_flag)
        initial_learning_rate = initial_lr
        final_learning_rate = 0.03

        # Define learning rate scheduler
        lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
            initial_learning_rate=initial_lr,
            decay_steps=16,
            alpha=final_learning_rate / initial_learning_rate
        )

        # Adam optimizer with the cosine scheduler
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

        # Compile the model
        model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy', 'AUC']
        )

        lr_logger = LearningRateLogger()

        # Train the model with the callback
        history = model.fit(
            train_images, train_labels,
            epochs=1,
            batch_size=32,
            verbose=1,
            validation_data=(test_images, test_labels),
            callbacks=[lr_logger]
        )

        # Update the learning rate for the next iteration
        if lr_logger.new_lr:
            initial_lr = lr_logger.new_lr

        # Store losses and metrics for this iteration
        seed_losses.append(history.history['loss'][0])
        seed_val_losses.append(history.history['val_loss'][0])
        seed_train_acc.append(history.history['accuracy'][0])
        seed_val_acc.append(history.history['val_accuracy'][0])
        seed_train_auc.append(history.history['AUC'][0])  # Add training AUC value
        seed_val_auc.append(history.history['val_AUC'][0])  # Add validation AUC value

        # Save the weights after training
        model.save_weights('/home/HardDisk/quang_nguyen/quantum_ml/experiment_dropout/quantum_weights3.weights.h5')

    # Store the metrics for the seed
    seeds.append(random)
    all_losses.append(seed_losses)
    all_val_losses.append(seed_val_losses)
    all_train_acc.append(seed_train_acc)
    all_val_acc.append(seed_val_acc)
    all_train_auc.append(seed_train_auc)  # Store training AUC
    all_val_auc.append(seed_val_auc)  # Store validation AUC

accuracies = [max(sublist) for sublist in all_val_acc]
aucs = [max(sublist) for sublist in all_val_auc]


# Calculate average metrics and 95% confidence intervals
mean_accuracy = np.mean(accuracies)
ci_accuracy_lower = np.percentile(accuracies, 2.5)
ci_accuracy_upper = np.percentile(accuracies, 97.5)

mean_auc = np.mean(aucs)
ci_auc_lower = np.percentile(aucs, 2.5)
ci_auc_upper = np.percentile(aucs, 97.5)

# Print results
print(f"Estimated Accuracy: {mean_accuracy:.4f}")
print(f"95% Confidence Interval for Accuracy: [{ci_accuracy_lower:.4f}, {ci_accuracy_upper:.4f}]")
print(f"Estimated AUC: {mean_auc:.4f}")
print(f"95% Confidence Interval for AUC: [{ci_auc_lower:.4f}, {ci_auc_upper:.4f}]")


Training with random seed: 10
Epochs 1
Not Applying Quantum Dropout
34/34 ━━━━━━━━━━━━━━━━━━━━ 18s 137ms/step - AUC: 0.5369 - accuracy: 0.5336 - loss: 0.5103 - val_AUC: 0.6792 - val_accuracy: 0.6250 - val_loss: 0.6406
Epochs 2
Not Applying Quantum Dropout
34/34 ━━━━━━━━━━━━━━━━━━━━ 17s 140ms/step - AUC: 0.6320 - accuracy: 0.6308 - loss: 0.4824 - val_AUC: 0.7274 - val_accuracy: 0.6675 - val_loss: 0.6161
Epochs 3
Not Applying Quantum Dropout
34/34 ━━━━━━━━━━━━━━━━━━━━ 19s 120ms/step - AUC: 0.5801 - accuracy: 0.5746 - loss: 0.3983 - val_AUC: 0.6081 - val_accuracy: 0.6100 - val_loss: 0.6914
Epochs 4
Not Applying Quantum Dropout
34/34 ━━━━━━━━━━━━━━━━━━━━ 19s 128ms/step - AUC: 0.6057 - accuracy: 0.5988 - loss: 0.5292 - val_AUC: 0.6419 - val_accuracy: 0.6300 - val_loss: 0.6656
Epochs 5
Not Applying Quantum Dropout
34/34 ━━━━━━━━━━━━━━━━━━━━ 21s 135ms/step - AUC: 0.6140 - accuracy: 0.6216 - loss: 0.4358 - val_AUC: 0.7660 - val_accuracy: 0.7175 - val_loss: 0.5765
Epochs 6
Applying Quantum Drop

# Plot Val Loss and Train Loss

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf


all_train_auc = []  # List to store training AUC for plotting
all_val_auc = []  # List to store validation AUC for plotting
all_losses = []  # For training loss
all_val_losses = []  # For validation loss
all_train_acc = []  # For training accuracy
all_val_acc = []  # For validation accuracy
seeds = []

for random in [10]:
    if os.path.exists('/home/HardDisk/quang_nguyen/quantum_ml/experiment_dropout/quantum_weights.weights.h5'):
        os.remove('/home/HardDisk/quang_nguyen/quantum_ml/experiment_dropout/quantum_weights.weights.h5')
    rd.seed(random)
    np.random.seed(random)
    tf.random.set_seed(random)
    qml.numpy.random.seed(random)
    print('Training with random seed:', random)
    initial_lr = 0.3
    seed_losses = []
    seed_val_losses = []
    seed_train_acc = []
    seed_val_acc = []
    seed_train_auc = []  # List to store training AUC for each epoch
    seed_val_auc = []  # List to store validation AUC for each epoch

    for iteration in range(1, 30):
        print(f"Epochs {iteration}")

        # Switch dropout flag randomly
        if rd.random() <= 0.5 and iteration != 1:
            drop_flag = True
            print("Applying Quantum Dropout")
        else:
            drop_flag = False
            print("Not Applying Quantum Dropout")

        # Create the model
        model = HybridModel(apply_quantum_dropout=drop_flag)
        initial_learning_rate = initial_lr
        final_learning_rate = 0.03

        # Define learning rate scheduler
        lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
            initial_learning_rate=initial_lr,
            decay_steps=16,
            alpha=final_learning_rate / initial_learning_rate
        )

        # Adam optimizer with the cosine scheduler
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

        # Compile the model
        model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy', 'AUC']
        )

        lr_logger = LearningRateLogger()

        # Train the model with the callback
        history = model.fit(
            train_images, train_labels,
            epochs=1,
            batch_size=32,
            verbose=1,
            validation_data=(test_images, test_labels),
            callbacks=[lr_logger]
        )

        # Update the learning rate for the next iteration
        if lr_logger.new_lr:
            initial_lr = lr_logger.new_lr

        # Store losses and metrics for this iteration
        seed_losses.append(history.history['loss'][0])
        seed_val_losses.append(history.history['val_loss'][0])
        seed_train_acc.append(history.history['accuracy'][0])
        seed_val_acc.append(history.history['val_accuracy'][0])
        seed_train_auc.append(history.history['AUC'][0])  # Add training AUC value
        seed_val_auc.append(history.history['val_AUC'][0])  # Add validation AUC value

        # Save the weights after training
        model.save_weights('/home/HardDisk/quang_nguyen/quantum_ml/experiment_dropout/quantum_weights.weights.h5')

    # Store the metrics for the seed
    seeds.append(random)
    all_losses.append(seed_losses)
    all_val_losses.append(seed_val_losses)
    all_train_acc.append(seed_train_acc)
    all_val_acc.append(seed_val_acc)
    all_train_auc.append(seed_train_auc)  # Store training AUC
    all_val_auc.append(seed_val_auc)  # Store validation AUC

# Plotting for each seed
epochs = range(1, 30)

for i, seed in enumerate(seeds):
    plt.figure(figsize=(12, 12))

    # Loss plot for the current seed
    plt.subplot(3, 1, 1)
    plt.plot(epochs, all_losses[i], label='Train Loss')
    plt.plot(epochs, all_val_losses[i], linestyle='--', label='Test Loss')
    plt.title(f'Loss per Epoch (Seed {seed})')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()

    # Accuracy plot for the current seed
    plt.subplot(3, 1, 2)
    plt.plot(epochs, all_train_acc[i], label='Train Acc')
    plt.plot(epochs, all_val_acc[i], linestyle='--', label='Val Acc')
    plt.title(f'Accuracy per Epoch (Seed {seed})')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()

    # AUC plot for the current seed
    plt.subplot(3, 1, 3)
    plt.plot(epochs, all_train_auc[i], label='Train AUC')
    plt.plot(epochs, all_val_auc[i], linestyle='--', label='Val AUC')
    plt.title(f'AUC per Epoch (Seed {seed})')
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.legend()
    plt.grid()

    plt.tight_layout()
    plt.show()


Training with random seed: 10
Epochs 1
Not Applying Quantum Dropout
34/34 ━━━━━━━━━━━━━━━━━━━━ 21s 139ms/step - AUC: 0.5369 - accuracy: 0.5336 - loss: 0.5103 - val_AUC: 0.6787 - val_accuracy: 0.6250 - val_loss: 0.6408
Epochs 2
Not Applying Quantum Dropout
34/34 ━━━━━━━━━━━━━━━━━━━━ 20s 132ms/step - AUC: 0.6320 - accuracy: 0.6308 - loss: 0.4824 - val_AUC: 0.7274 - val_accuracy: 0.6675 - val_loss: 0.6161
Epochs 3
Not Applying Quantum Dropout
34/34 ━━━━━━━━━━━━━━━━━━━━ 19s 132ms/step - AUC: 0.5902 - accuracy: 0.5839 - loss: 0.3965 - val_AUC: 0.7236 - val_accuracy: 0.6775 - val_loss: 0.6134
Epochs 4
Not Applying Quantum Dropout
34/34 ━━━━━━━━━━━━━━━━━━━━ 18s 139ms/step - AUC: 0.6238 - accuracy: 0.6063 - loss: 0.5333 - val_AUC: 0.7298 - val_accuracy: 0.6775 - val_loss: 0.6102
Epochs 5
Not Applying Quantum Dropout
34/34 ━━━━━━━━━━━━━━━━━━━━ 17s 142ms/step - AUC: 0.6140 - accuracy: 0.6216 - loss: 0.4358 - val_AUC: 0.7660 - val_accuracy: 0.7175 - val_loss: 0.5765
Epochs 6
Applying Quantum Drop